<h1>下載中文維基數據</h1>

<p>請前往維基百科:資料庫下載按日期來挑選更新的訓練資料。( 請挑選以pages-articles.xml.bz2為結尾的檔案 )</p>

https://dumps.wikimedia.org/zhwiki/

In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [1]:
import sys

from gensim.corpora import WikiCorpus

wikifile = 'zhwiki-20171020-pages-articles.xml.bz2'

wiki_corpus = WikiCorpus(wikifile, dictionary={})
texts_num = 0

logging.info('Step1-開始輸出wiki_texts.txt')
with open("wiki_texts.txt",'w',encoding='utf-8') as output:
    for text in wiki_corpus.get_texts():
        output.write(' '.join(text) + '\n')
        texts_num += 1
        if texts_num % 10000 == 0:
            logging.info("已處理 %d 篇文章" % texts_num)

2017-10-26 00:40:09,071 : INFO : Step1-開始輸出wiki_texts.txt
2017-10-26 00:40:39,070 : INFO : 已處理 10000 篇文章
2017-10-26 00:41:03,042 : INFO : 已處理 20000 篇文章
2017-10-26 00:41:24,913 : INFO : 已處理 30000 篇文章
2017-10-26 00:41:46,827 : INFO : 已處理 40000 篇文章
2017-10-26 00:42:09,792 : INFO : 已處理 50000 篇文章
2017-10-26 00:42:33,099 : INFO : 已處理 60000 篇文章
2017-10-26 00:42:54,347 : INFO : 已處理 70000 篇文章
2017-10-26 00:43:16,815 : INFO : 已處理 80000 篇文章
2017-10-26 00:43:38,681 : INFO : 已處理 90000 篇文章
2017-10-26 00:44:02,444 : INFO : 已處理 100000 篇文章
2017-10-26 00:44:31,658 : INFO : 已處理 110000 篇文章
2017-10-26 00:44:58,248 : INFO : 已處理 120000 篇文章
2017-10-26 00:45:25,169 : INFO : 已處理 130000 篇文章
2017-10-26 00:45:54,113 : INFO : 已處理 140000 篇文章
2017-10-26 00:46:23,436 : INFO : 已處理 150000 篇文章
2017-10-26 00:46:50,619 : INFO : 已處理 160000 篇文章
2017-10-26 00:47:21,180 : INFO : 已處理 170000 篇文章
2017-10-26 00:47:51,502 : INFO : 已處理 180000 篇文章
2017-10-26 00:50:13,195 : INFO : 已處理 190000 篇文章
2017-10-26 00:51:10,938 : INFO : 已處理 20

<h1>使用 OpenCC 將維基文章統一轉換為繁體中文</h1>

In [2]:
import os
os.system("opencc -i wiki_texts.txt -o wiki_zh_tw.txt -c s2tw.json")

0

<h1>使用jieba 對文本斷詞，並去除停用詞</h1>

In [2]:
# -*- coding: utf-8 -*-

import jieba
import logging

# jieba custom setting.
jieba.set_dictionary('jieba_dict/dict.txt.big')

# load stopwords set
stopwordset = set()
with open('jieba_dict/stopwords.txt','r',encoding='utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))

texts_num = 0

output = open('wiki_seg.txt','w')
with open('wiki_zh_tw.txt','r') as content :
    for line in content:
        line = line.strip('\n')
        words = jieba.cut(line, cut_all=False)
        for word in words:
            if word not in stopwordset:
                output.write(word +' ')

        texts_num += 1
        if texts_num % 10000 == 0:
            logging.info("已完成前 %d 行的斷詞" % texts_num)
output.close()
logging.info('已完成')

Building prefix dict from /Users/dennislin/Documents/python/ChineseNLP/gensim/jieba_dict/dict.txt.big ...
2017-10-27 00:01:17,203 : DEBUG : Building prefix dict from /Users/dennislin/Documents/python/ChineseNLP/gensim/jieba_dict/dict.txt.big ...
Loading model from cache /var/folders/xk/jzfb6tps7kz7xv_j3tjr0h7c0000gn/T/jieba.u7727492867168ca4ad16b9668334ced9.cache
2017-10-27 00:01:17,206 : DEBUG : Loading model from cache /var/folders/xk/jzfb6tps7kz7xv_j3tjr0h7c0000gn/T/jieba.u7727492867168ca4ad16b9668334ced9.cache
Loading model cost 1.558 seconds.
2017-10-27 00:01:18,763 : DEBUG : Loading model cost 1.558 seconds.
Prefix dict has been built succesfully.
2017-10-27 00:01:18,764 : DEBUG : Prefix dict has been built succesfully.
2017-10-27 00:04:18,564 : INFO : 已完成前 10000 行的斷詞
2017-10-27 00:06:30,138 : INFO : 已完成前 20000 行的斷詞
2017-10-27 00:09:09,825 : INFO : 已完成前 30000 行的斷詞
2017-10-27 00:12:03,769 : INFO : 已完成前 40000 行的斷詞
2017-10-27 00:13:49,519 : INFO : 已完成前 50000 行的斷詞
2017-10-27 00:15:49

<h1>使用gensim 的 word2vec 模型進行訓練</h1>

In [ ]:
from gensim.models import word2vec

sentences = word2vec.Text8Corpus("wiki_seg.txt")
model = word2vec.Word2Vec(sentences, size=250)

#保存模型，供日後使用
model.save("med250.model.bin")
logging.info('已完成')

<h1>測試模型</h1>

In [ ]:
from gensim.models import word2vec
from gensim import models
import logging

model = models.Word2Vec.load('med250.model.bin')

print("提供 3 種測試模式\n")
print("輸入一個詞，則去尋找前一百個該詞的相似詞")
print("輸入兩個詞，則去計算兩個詞的餘弦相似度")
print("輸入三個詞，進行類比推理")

while True:
    try:
        query = input()
        q_list = query.split()

        if len(q_list) == 1:
            print("相似詞前 100 排序")
            res = model.most_similar(q_list[0],topn = 100)
            for item in res:
                print(item[0]+","+str(item[1]))

        elif len(q_list) == 2:
            print("計算 Cosine 相似度")
            res = model.similarity(q_list[0],q_list[1])
            print(res)
        else:
            print("%s之於%s，如%s之於" % (q_list[0],q_list[2],q_list[1]))
            res = model.most_similar([q_list[0],q_list[1]], [q_list[2]], topn= 100)
            for item in res:
                print(item[0]+","+str(item[1]))
        print("----------------------------")
    except Exception as e:
        print(repr(e))

提供 3 種測試模式

輸入一個詞，則去尋找前一百個該詞的相似詞
輸入兩個詞，則去計算兩個詞的餘弦相似度
輸入三個詞，進行類比推理
漫威
相似詞前 100 排序
夜魔俠,0.6886358261108398
marvel,0.6717044115066528
水行俠,0.6563444137573242
神奇四俠,0.6440639495849609
蜘蛛人,0.6250507235527039
死侍,0.6121244430541992
復仇者,0.571926474571228
蝙蝠俠,0.549408495426178
皮克斯,0.5481370687484741
綠箭俠,0.544106125831604
蜘蛛俠,0.543834388256073
孩之寶,0.5374860167503357
制裁者,0.529426097869873
閃電俠,0.5280251502990723
浩克,0.5191125273704529
deadpool,0.5160672664642334
漫畫書,0.5153291821479797
daredevil,0.5102291107177734
前傳,0.5067425966262817
batman,0.5054164528846741
派拉蒙,0.5031026601791382
戰警,0.5023834705352783
懲罰者,0.4980250597000122
奇航記,0.49494341015815735
霹靂遊俠,0.49482306838035583
強斯,0.4927905797958374
樂高,0.49158456921577454
dcau,0.49081557989120483
惡棍,0.4886305630207062
女超人,0.488104909658432
夢工廠,0.4874275326728821
夜翼,0.48529163002967834
綠巨人,0.48446789383888245
robotech,0.48386719822883606
溫斯坦,0.4834345877170563
返校日,0.48289328813552856
星際,0.48150044679641724
小勞勃,0.48102912306785583
獅門,0.4810125231742859
電子遊